### Ladder of Causation

A metaphor for understanding the distinct levels of relationships between variables. The ladder has 3 rungs and each rung is related to different activity and offers answers to different types of causal questions. Each rung also comes with a distinct set of mathematical tools.

1. <u><b>Association</b></u>: The activity related to this level is <u>observing</u>. Using association, we can answer questions about how seeing one thing changes our belief about another thing. E.g. how observing a successful product launch changes our belief that the company's stock price will go up.
2. <u><b>Intervention</b></u>: We can intervene on one variable to check how it influences some other variable. E.g. if I go to bed earlier, will I have more energy the following morning. 
3. <u><b>Counterfactual reasoning</b></u>: Imagining/understanding. What would have happened had we done something differently. E.g. would I have made it to the office on time, had I taken the train rather than the car?

#### Associations

We can quantify associational relationships using <u>conditional probability</u>. E.g. what is the probability that a person will buy book A, given they've bought book B: P(book A|book B).

This question does not give us any information on the causal relationship between both events. We don't know if buying book B caused the customer to buy book A, vice versa, or if there is some other (unobserved) event that caused both. We only get information about non-causal association between these events. 

<b><u>Structural Causal Models (SCMs)</u></b> are a simple tool to encode causal relationships between variables. We use them as our <u>data-generating process</u>. After generating the data, we will pretend to forget what our SCM was in order to mimic a frequent real-world scenario where the true data-generating process is unknown and the only thing we have is observational data. Code an example:

U<sub>0</sub> ~ U(0,1) (a continuous random variable uniformly distributed between 0 and 1)  
U<sub>1</sub> ~ N(0,1) (a normally distributed random variable with a mean value of 0 and a standard deviation of 1)  
A := 1<sub>{U<sub>0</sub>>0.61}</sub> (a binary variable)  
B := 1<sub>{(A + 0.5 * U<sub>1</sub>)>0.2}</sub> (a binary variable)

In [1]:
import numpy as np
from scipy import stats

In [2]:
class BookSCM:
    def __init__(self, random_seed=None):
        self.random_seed = random_seed
        self.u_0 = stats.uniform()
        self.u_1 = stats.norm()

    def sample(self, sample_size=100):
        "Samples from the SCM"
        u_0 = self.u_0.rvs(sample_size)
        u_1 = self.u_1.rvs(sample_size)
        a = u_0 > 0.61
        b = (a + 0.5 * u_1) > 0.2
        
        return a, b

In [3]:
# Instantiate SCM and set random seed
scm = BookSCM(random_seed=42)

# Sample 100 samples from it
buy_book_a, buy_book_b = scm.sample(100)

# Check whether shapes are as expected
print(f"Book a shape: {buy_book_a.shape}")
print(f"Book b shape: {buy_book_b.shape}")

Book a shape: (100,)
Book b shape: (100,)


In [4]:
# Compute the conditional probability P(book A|book B)
proba_book_a_given_book_b = buy_book_a[buy_book_b].sum() / buy_book_a[buy_book_b].shape[0]

print(f"Probability of buying book A given B: {proba_book_a_given_book_b:0.3f}")

Probability of buying book A given B: 0.567


A 56.7% probability of buying book A, given we bought book B. This indicates a positive relationship between both variables (if there was no association between them, we would expect the result to be 39%).
- If A and B were independent (not associated), then knowing B = 1 wouldn't change the chance of A = 1.
- So the conditional probability P(A=1|B=1) would simply equal the marginal probability P(A=1).
- The marginal probability of A = 0.39 = 39%, given how A is defined: P(A=1) = P(U<sub>0</sub>>0.61)=1 = (1 - 0.61) = 0.39.

The result tells us that we can make meaningful predictions using observations alone. This is at the core of most supervised ML models. Associations can have high practical significance in the absence of knowledge of the data-generating process. Conditional probability allowed us to draw conclusions based on the observational data alone.

#### Interventions

We change one thing in the world and observe whether and how this change affects another thing in the world. To describe interventions mathematically, we use a <i>do</i> operator, like so: P(Y = 1|<sub>do</sub>(X = 0)).

The above states that the probability of Y = 1, given that we set X to 0. That we change the value of X is critical here - this is the inherent difference between intervening and conditioning (the operation used to obtain conditional probabilities). Conditioning only modifies our view of the data, while intervening affects the distribution by actively setting one (or more) variable(s) to a fixed value (or a distribution). Intervention changes the system, but conditioning does not. 

For this example:
U<sub>0</sub> ~ N(0,1)  
U<sub>1</sub> ~ N(0,1)  
A := U<sub>0</sub>  
B := 5A + U<sub>1</sub>

Here we set A and B to be continuous variables as opposed to binary. 

In [3]:
# Define sample size and set random seed
SAMPLE_SIZE = 100
np.random.seed(42)

# Build SCM
u_0 = np.random.randn(SAMPLE_SIZE)
u_1 = np.random.randn(SAMPLE_SIZE)
a = u_0
b = 5 * a + u_1

# Compute correlation coefficient between A and B
r, p = stats.pearsonr(a, b)

print(f"Mean of B before any intervention: {b.mean():.3f}")
print(f"Variance of B before any intervention: {b.var():.3f}")
print(f"Correlation between A and B:\nr = {r:.3f}; p = {p:.3f}\n")

Mean of B before any intervention: -0.497
Variance of B before any intervention: 20.144
Correlation between A and B:
r = 0.978; p = 0.000



The correlation between A and B is very high - not surprising since B is a linear function of A. The mean of B is slightly below 0, and the variance is around 20. Now let's intervene on A by fixing its value at 1.5.

In [4]:
a = np.array([1.5] * SAMPLE_SIZE)
b = 5 * a + u_1

print(f"Mean of B after intervention: {b.mean():.3f}")
print(f"Variance of B after intervention: {b.var():.3f}")

Mean of B after intervention: 7.522
Variance of B after intervention: 0.900


The intervention has changed the system - both the mean and the variance have changed. The value of our intervention on A is much bigger than we'd expect from our original distribution of A (centered at 0), while the variance has shrunk (because A became constant - the only remaining variability in B comes from its stochastic parante, U<sub>1</sub>). What happens if we intervene on B instead?

In [5]:
a = u_0
b = np.random.randn(SAMPLE_SIZE)

r, p = stats.pearsonr(a, b)

print(f"Mean of B after the intervention on B: {b.mean():.3f}")
print(f"Variance of B after the intervention on B: {b.var():.3f}")
print(f"Correlation between A and B after intervening on B:\nr = {r:.3f}; p = {p:.3f}\n")

Mean of B after the intervention on B: 0.065
Variance of B after the intervention on B: 1.164
Correlation between A and B after intervening on B:
r = 0.191; p = 0.057



The correlation dropped significantly and the p-value indicates a lack of significance. This indicates that after the intervention, A and B became (linearly) independent. The result suggests no causal link from B to A, whereas the previous result demonstrated a causal link from A to B. 

Remember: <u>correlation doesn't equal causation!</u> A correlation coefficient between X and Y might be 0, if their relationship is not monotonic. Popular correlation metrics (Pearson's, Spearman's) cannot capture monotonic relationships. So, a lack of traditional correlation does not imply independence between variables.

#### Counterfactuals

We can think of counterfactuals as a minimal modification to a system. In this sense, they are similar to interventions. But counterfactuals can be thought of as a hypothetical or simulated intervention that assume a particular state of the world (interventions do not make assumptions about the state of the world.) E.g., answering a counterfactual question such as "Would Sophie have bought the chocolate last Friday had he seen the advert last week" requires us to know many things about Sophie (and her environment) in the past. Counterfactuals can never be observed.

Furthermore, two counterfactual causal models can lead to the same interventional distribution. E.g. for both models, the causal effect of a drug is equal to 0, but in one model no patient is affected by the drug and in the second model all patients are affected by the drug. So, counterfactual outcomes differ for both models. 

<u>Example</u>: You had a coffee this morning and now you have a stomach ache. Would you have felt the same or better if you hadn't had your coffee? Note that you cannot answer this question with interventions. A randomised experiment would only be able to answer a question such as:  
- "What the is the probability that people similar to you react to coffee the way you reacted, given similar circumstances?"  

Counterfactuals are trying to answer a different question:  
- "Given an alternative world that is identical to ours and only differs in the fact that you did not drink coffee this morning, what is the probability that you'd have a stomach ache?"

Let's formalise it. The fact that you drank coffee this morning can be denoted by X = 1, and that you feel ill as Y<sub>X = 1</sub> = 1. The quantity that we estimate is therefore: P(Y<sub>X = 0</sub> = 1 | X = 1, Y<sub>X = 1</sub> = 1). This is the probability that you'd feel ill if you hadn't had coffee, where:
- P(Y<sub>X = 0</sub> = 1) stands for the probability that you'd feel ill (in the alternative world) if you hadn't had coffee;
- X = 1 denotes that you had coffee in the real world;
- Y<sub>X = 1</sub> = 1 says that you had coffee and feel ill (in the real world).

<b>Computing counterfactuals</b>: This requires an SCM that is fully specified, at least for all the relevant variables. This means we need to have full knowledge about functions that relate to relevant variables in the SCM and full knowledge of all relevant exogenous variables in a system. Step by step:
- <u>Abduction</u>: using evidence to calculate values of exogenous variables
- <u>Modification</u>: replacing the structural equation for the treatment with a counterfactual value
- <u>Prediction</u>: using the modified SCM to compute the new value of the outcome under the counterfactual.

For the coffee example - T denotes the treatment, drinking coffee, while U characterises you fully as an individual in our simplied world. U = 1 stands for coffee sensitivity, and U = 0 for lack thereof. Let's also assume we know the causal mechanism for reaction to coffee, defined by the following SCM:
- T := t
- Y := TU + (T - 1)(U - 1)

We know the outcoe under the actual treatment, Y<sub>T = 1</sub> = 1 (you drank the coffee and feel ill), but we don't know your characteristics. Can we do something about it? Yes! Our model lets us unambiguously deduct the value of U (denoted as u), given we know the values of Y and T by solving the structural equation for Y:
- u = $\frac{T + Y - 1}{2T - 1}$
- u = $\frac{1 + 1 - 1}{2*1 - 1}$ = $\frac{2 - 1}{2 - 1}$ = 1

The value we obtained for U reveals that you are a coffee-sensitive person. This step (solving for values of exogenous variables U) is called <b>abduction</b>. Now, we have all the elements necessary to compute counterfactual outcomes at our disposal - a causal model and knowledge of your personal characteristics. We now use <b>modification</b> to fix the value of our treatment at the counterfactual of interest, T = 0:
- T := 0
- Y := 0U + (0 - 1)(U - 1)

Then, the last step is <b>prediction</b>, where we substitute U with the value we computed before:
- Y := 0*1 + (0 - 1)(1 - 1) = 0

Therefore, you wouldn't feel bad if you hadn't had your coffee.

In [2]:
# Counterfactual SCM class with three methods corresponding to three steps of counterfactual computation
class CounterFactualSCM:

    # Computes the value of U, given values for treatment & actual outcome
    def abduct(self, t, y):
        return (t + y - 1) / (2*t - 1)
    
    # Modifies the SCN by assigning t to T
    def modify(self, t):
        return lambda u: t * u + (t - 1) * (u - 1)
    
    # Takes the modified SCM and generates a counterfactual prediction by assigning the actual value of U to the new SCM
    def predict(self, u, t):
        return self.modify(t)(u)

In [4]:
# Instantiate SCM, assign the known treatment and outcome values to t and y respectively
coffee = CounterFactualSCM()
t = 1
y = 1

# Obtain U by performing abduction
u = coffee.abduct(t=t, y=y)
print(f"u = {u}")

# Compute the counterfactual prediction
print(
    f"Counterfactual prediction: {coffee.predict(u=u, t=0)}"
)

u = 1.0
Counterfactual prediction: 0.0
